In [1]:
from  dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
import operator
from typing import Annotated, TypedDict, List
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [4]:
class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]

In [15]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, path_map={
            True: "action", False: END
        })
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]} 

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!!!")
        return {"messages": results}

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

In [16]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt)

In [17]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_JYSwa4exkJSVbCLFEScM6LSK'}
Back to the model!!!


In [18]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JYSwa4exkJSVbCLFEScM6LSK', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c5621672-89aa-4808-8fae-bdecd29fe102-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_JYSwa4exkJSVbCLFEScM6LSK'}], usage_metadata={'input_tokens': 151, 'output_tokens': 22, 'total_tokens': 173}),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 

In [19]:
result["messages"][-1].content

"The current weather in San Francisco is as follows:\n\n- **Temperature**: 9.0°C (48.2°F)\n- **Condition**: Mist\n- **Wind**: 2.2 mph (3.6 kph) from the North\n- **Humidity**: 100%\n- **Visibility**: 4.8 km (2.0 miles)\n- **Pressure**: 1012.0 mb (29.88 in)\n- **Cloud Cover**: 25%\n- **Feels Like**: 7.6°C (45.6°F)\n- **UV Index**: 4 (Moderate)\n\n![Weather Icon](//cdn.weatherapi.com/weather/64x64/day/143.png)\n\nFor more detailed and continuous updates, you can visit [WeatherAPI](https://www.weatherapi.com/) or [NOAA's San Francisco Weather Page](https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA)."

In [20]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_wpoZHtbVhgYsuMszTAPlOCFY'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_pmGVqKseoIbKJr7bne1DLciV'}
Back to the model!!!


In [23]:
result["messages"][-1].content

'### Current Weather in San Francisco:\n- **Temperature**: 9.0°C (48.2°F)\n- **Condition**: Mist\n- **Wind**: 2.2 mph (3.6 kph) from the North\n- **Humidity**: 100%\n- **Visibility**: 4.8 km (2.0 miles)\n\n### Current Weather in Los Angeles:\n- **Temperature**: 16.7°C (62.1°F)\n- **Condition**: Overcast\n- **Wind**: 2.2 mph (3.6 kph) from the North\n- **Humidity**: 96%\n- **Visibility**: 11.0 km (6.0 miles)'

In [24]:
query = "Who won the superbowl in 2024? What is the GDP of that state?"
messages = [HumanMessage(content=query)]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_twSVkBPVHwLRWXQJf8EzV6f8'}
Back to the model!!!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2023'}, 'id': 'call_Ud871ZmlwyxAbUy66UMqcRnf'}
Back to the model!!!


In [25]:
result["messages"][-1].content

'The Kansas City Chiefs won the Super Bowl in 2024.\n\nAs of the 3rd quarter of 2023, the real GDP for the state of Missouri was approximately $423.6 billion.'